In [478]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster

from folium import plugins
from argopy import IndexFetcher as ArgoIndexFetcher
import numpy as np
import cartopy
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import matplotlib

import branca
from folium.features import DivIcon


In [479]:
def get_geojson_grid(upper_right, lower_left, n_lat=6, n_lon=6):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n_lat+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n_lon+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    coord_grids = []
    
    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]
            limits = {'lat': (lat, lat+lat_stride), 
                      'long': (lon, lon+lon_stride)}
            coord_grids.append(limits)

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)
            

    return coord_grids, all_boxes



In [480]:
# https://scitools.org.uk/cartopy/docs/latest/matplotlib/advanced_plotting.html
# https://matplotlib.org/basemap/api/basemap_api.html

In [481]:
latlong_nyc = (40, -70)
latlong_ocean = (20, -50)
# 5x5 grid -> (4, 4) grid

In [487]:
df_buoy = pd.read_pickle('atlantic_ocean_[-80,-39,0,50,2020-06-01,2021-07].pkl')

In [488]:
df_buoy

,file,date,longitude,latitude,ocean,profiler_code,institution_code,date_update,wmo,institution,profiler
0,coriolis/6902957/profiles/R6902957_093.nc,2020-06-01 03:18:00,-57.714,7.773,A,841,IF,2020-11-25 17:04:17,6902957,"Ifremer, France","Provor, Seabird conductivity sensor"
1,bodc/6901215/profiles/R6901215_182.nc,2020-06-01 03:39:26,-70.597,22.517,A,849,BO,2020-08-01 23:41:24,6901215,"BODC, United Kingdom",Apex-D deep float
2,aoml/4903036/profiles/R4903036_091.nc,2020-06-01 03:53:26,-67.636,34.730,A,854,AO,2021-03-01 18:13:58,4903036,"AOML, USA",S2A float
3,aoml/4902121/profiles/R4902121_146.nc,2020-06-01 05:06:39,-72.835,37.655,A,854,AO,2021-03-02 01:02:47,4902121,"AOML, USA",S2A float
4,aoml/4903049/profiles/R4903049_060.nc,2020-06-01 05:16:58,-67.700,35.733,A,854,AO,2020-06-11 04:00:36,4903049,"AOML, USA",S2A float
...,...,...,...,...,...,...,...,...,...,...,...
7667,aoml/4903252/profiles/R4903252_147.nc,2021-06-08 21:53:22,-73.036,36.613,A,854,AO,2021-06-09 00:00:39,4903252,"AOML, USA",S2A float
7668,aoml/4903342/profiles/R4903342_010.nc,2021-06-09 00:51:59,-52.728,16.493,A,854,AO,2021-06-09 02:00:48,4903342,"AOML, USA",S2A float
7669,aoml/4902348/profiles/R4902348_181.nc,2021-06-09 01:31:55,-60.736,34.116,A,854,AO,2021-06-09 03:00:26,4902348,"AOML, USA",S2A float
7670,aoml/4903051/profiles/R4903051_085.nc,2021-06-09 01:42:31,-61.997,38.309,A,854,AO,2021-06-09 03:00:29,4903051,"AOML, USA",S2A float


In [489]:
print("Current date and time: ", datetime.datetime.now())
df_buoy['longlat'] = list(zip(df_buoy.longitude, df_buoy.latitude)) 
df_timelapse = df_buoy[['longlat', 'date', 'wmo']]

Current date and time:  2021-06-12 21:55:03.478284


In [490]:
TIME_START = '2021-01-01'
TIME_END = '2021-01-31'

In [491]:
timed_df = df_buoy[ (df_buoy.date >= TIME_START) & (df_buoy.date <= TIME_END)]
# DO NOT USE DATE_UPDATE!

In [ ]:
print(coord_grids)

In [493]:

# http://www.garagegames.com/community/blogs/view/309
lower_left = np.array([20, -70])
diff = 21
upper_right = lower_left + diff

n = 7

lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

lat_stride = lat_steps[1] - lat_steps[0]
lon_stride = lon_steps[1] - lon_steps[0]

zeros = [] # used for AABB(axis aligned bounding box) line intersection check later

profile_counts_grid = []
for lat in lat_steps[:-1]:
    for lon in lon_steps[:-1]:
        itfits = timed_df[ 
            (timed_df['latitude'] >= lat ) &
                 (timed_df['latitude'] < lat + lat_stride) &
            (timed_df['longitude'] >= lon ) &
                 (timed_df['longitude'] < lon + lon_stride) 
               ]
        #print(itfits.shape, lon, lat)
        profile_counts_grid.append(itfits.shape[0])
        if itfits.shape[0] == 0: # There were no profiles in this lat long
            #zeros.append((lat, lat+lat_stride, lon, lon+lon_stride))
            zeros.append( shapely.geometry.box(lat, lat+lat_stride, lon, lon+lon_stride))
print(timed_df.shape)
        

(609, 12)


In [494]:
lower_left = [20, -70]
upper_right = [40, -50]
m = folium.Map(zoom_start = 4, location=[30, -60], width=600, height=400)
# ULeft, URight, LLeft, LRight

coord_grids, boxes = get_geojson_grid(upper_right, lower_left , n_lat=7, n_lon=7)

colormap = branca.colormap.linear.YlOrRd_09.scale(0, 30)
colormap = colormap.to_step(index=[0, 1, 2, 5, 10, 15, 20, 25, 30])
colormap.caption = 'Number of profiles'
colormap.add_to(m)

for counts, geo_json in zip(profile_counts_grid, boxes):

    if counts == 0:
        color = 'b'
        color = mpl.colors.to_hex(color)
    else:
        #color = plt.cm.Oranges(counts / len(boxes))
        #color = mpl.colors.to_hex(color)
        color = colormap(counts)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, 
                        color=color: {
                            'fillColor': color,
                            'color':"black",
                            'weight': 0.4,
                            'dashArray': '5, 5',
                            'fillOpacity': 0.70,
                        })
    popup = folium.Popup(f"Counts:{counts}")
    gj.add_child(popup)

    m.add_child(gj)
    
folium.map.Marker(
    [40,- 48],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 14pt"><b>Heatmap</b><br>Num. of profiles per 3°x3°<br><br>From <font color="purple">{TIME_START}</font> to '
        f'<font color="purple">{TIME_END}</font><br><br><font color="blue">Blue is 0 counts</font> </div>',
        )
    ).add_to(m)

m

In [498]:
colormap(1)

'#ffffccff'

# Check for intersections

In [499]:
import warnings 
#warnings.filterwarnings(action='once')
warnings.filterwarnings('ignore')

In [500]:
import shapely
from shapely.geometry import * 
line = LineString([(0.5, 0.5), (1.5, 1)])
b = box(0.0, 0.0, 1.0, 1.0) # minx, miny, maxx, maxy
b.intersects(line)

True

In [501]:
df_buoy = pd.read_pickle('atlantic_ocean.pkl')
list_buoy_ids = df_buoy.wmo.unique()
len(list_buoy_ids)

df_buoy = df_buoy.dropna(subset=['latitude', 'longitude'])
df_buoy.sort_values(by=['wmo', 'date'])
df_buoy['longlat'] = list(zip(df_buoy.longitude, df_buoy.latitude)) 
df_timelapse = df_buoy[['longlat', 'date', 'wmo']]

FileNotFoundError: [Errno 2] No such file or directory: 'atlantic_ocean.pkl'

In [502]:
# Add collision checking code
# for each "skipped" cell - 
# oh dear we actually have 3d dimesions, 2d + time ... anyway
# simplest: only look at lines that begin and end within the timeframe.

list_buoy_lines = []

NaT_fill = 0 
gps_fill = -999
    
for ith_buoy, buoy in enumerate(list_buoy_ids):
    # Create timelapse lines for a single buoy
    buoy_df = df_timelapse[ (df_timelapse.wmo == buoy) & (df_timelapse.date >= TIME_START) & (df_timelapse.date <= TIME_END)]
    # Create lines consist of 
    # (latlong_pointA), timeA to (latlong_pointB, timeB)
    buoy_df['line_coords'] = list(zip(
        buoy_df['longlat'].shift(fill_value = gps_fill), 
        buoy_df['longlat']))
    
    buoy_df = buoy_df.sort_values(by=['wmo','date'])
    
    buoy_df['str_date'] = buoy_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    buoy_df['line_times'] = list(zip(
        buoy_df['str_date'].shift(fill_value = NaT_fill) , 
        buoy_df['str_date']
    ))
    buoy_df = buoy_df.iloc[1:]
    buoy_df['lineobject'] =  buoy_df['line_coords'].apply(lambda x: LineString(x))
    list_buoy_lines.append(buoy_df) # Due to use of pd.shift, first line is N/A
    
lines_df = pd.concat(list_buoy_lines)
    

In [123]:
lines_df
# TODO: need to be a little more careful about the timing truncation. 
# Currently: start and end points are all contained temporally. 
# So will NOT include lines that start before timeframe.
# Also will NOT include lines that end after timeframe.
# for now let's proceed

,longlat,date,wmo,line_coords,str_date,line_times,lineobject
5771,"(-70.526, 35.837)",2021-03-14 10:37:34,4903036,"((-70.402, 35.782), (-70.526, 35.837))",2021-03-14 10:37:34,"(2021-03-04 13:35:41, 2021-03-14 10:37:34)","LINESTRING (-70.402 35.782, -70.526 35.837)"
5964,"(-70.936, 35.466)",2021-03-24 07:27:00,4903036,"((-70.526, 35.837), (-70.936, 35.466))",2021-03-24 07:27:00,"(2021-03-14 10:37:34, 2021-03-24 07:27:00)","LINESTRING (-70.526 35.837, -70.93600000000001..."
5805,"(-62.445, 40.507)",2021-03-16 10:12:58,4902121,"((-61.837, 40.243), (-62.445, 40.507))",2021-03-16 10:12:58,"(2021-03-06 11:49:32, 2021-03-16 10:12:58)","LINESTRING (-61.837 40.243, -62.445 40.507)"
6003,"(-63.643, 40.662)",2021-03-26 08:36:57,4902121,"((-62.445, 40.507), (-63.643, 40.662))",2021-03-26 08:36:57,"(2021-03-16 10:12:58, 2021-03-26 08:36:57)","LINESTRING (-62.445 40.507, -63.643 40.662)"
5777,"(-57.835, 40.587)",2021-03-15 04:12:05,4903049,"((-58.287, 40.492), (-57.835, 40.587))",2021-03-15 04:12:05,"(2021-03-05 06:44:52, 2021-03-15 04:12:05)","LINESTRING (-58.287 40.492, -57.835 40.587)"
...,...,...,...,...,...,...,...
5815,"(-79.616, 25.304)",2021-03-16 17:34:18,4903252,"((-79.813, 24.866), (-79.616, 25.304))",2021-03-16 17:34:18,"(2021-03-11 20:03:29, 2021-03-16 17:34:18)","LINESTRING (-79.813 24.866, -79.616 25.304)"
5905,"(-79.601, 25.382)",2021-03-21 15:04:54,4903252,"((-79.616, 25.304), (-79.601, 25.382))",2021-03-21 15:04:54,"(2021-03-16 17:34:18, 2021-03-21 15:04:54)","LINESTRING (-79.616 25.304, -79.601 25.382)"
6008,"(-79.588, 25.54)",2021-03-26 12:39:05,4903252,"((-79.601, 25.382), (-79.588, 25.54))",2021-03-26 12:39:05,"(2021-03-21 15:04:54, 2021-03-26 12:39:05)","LINESTRING (-79.601 25.382, -79.58799999999999..."
5926,"(-49.633, 48.875)",2021-03-22 12:11:00,3901851,"((-49.797, 49.863), (-49.633, 48.875))",2021-03-22 12:11:00,"(2021-03-12 11:31:00, 2021-03-22 12:11:00)","LINESTRING (-49.797 49.863, -49.633 48.875)"


In [124]:
zeros

In [152]:
def create_line(row):
    line_feature =  { "type": "Feature", 
                "geometry": { "type": "LineString", 
                             "coordinates": row.line_coords, },
                "properties": { "times": row.line_times, 
                               "style": { "color": 'black', "weight": 3}, 
                               'icon': 'circle',
                               'iconstyle':{
                                   'fillColor': color,
                                   'opacity': 0.9,
                                   'stroke': False,
                                   'radius':5 
                               },
                               # Add click-to-popup 
                               'popup': f'Buoy ID: {row.wmo}, \n Date: {row.date}',#, Date uploaded: {row.date_update}',
                              },
                    }
    return line_feature

In [155]:
# sanity check - "date" = "Datetime" of a cycle, e.g. cycle 3, on the dataselection website
# compare to https://dataselection.euro-argo.eu/cycle/5013276
for box in zeros:
    print(list(box.exterior.coords))
    it_collides = []
    for i, row in lines_df.iterrows():
        if box.intersects(row['lineobject']):
            line_feature = create_line(row)
            #it_collides.append(row['lineobject'])
            it_collides.append(line_feature)
    print(len(it_collides))
#print(it_collides[0].xy)
print(it_collides[0])
    

[(-58.0, 26.0), (-58.0, -55.0), (23.0, -55.0), (23.0, 26.0), (-58.0, 26.0)]
104
[(-55.0, 29.0), (-55.0, -52.0), (26.0, -52.0), (26.0, 29.0), (-55.0, 29.0)]
106
[(-61.0, 32.0), (-61.0, -58.0), (29.0, -58.0), (29.0, 32.0), (-61.0, 32.0)]
163
{'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': ((-58.539, 17.512), (-58.577, 17.623))}, 'properties': {'times': ('2021-03-08 05:43:20', '2021-03-18 05:37:20'), 'style': {'color': 'black', 'weight': 3}, 'icon': 'circle', 'iconstyle': {'fillColor': '#fff2abff', 'opacity': 0.9, 'stroke': False, 'radius': 5}, 'popup': 'Buoy ID: 3901686, \n Date: 2021-03-18 05:37:20'}}


## Add intersecting buoys to heatmap

In [207]:
plugins.TimestampedGeoJson(
    { "type": "FeatureCollection", "features": it_collides},
    period="P1D", add_last_point=True, 
    duration = 'P2M',
    min_speed = 1,
    max_speed = 50,
    transition_time = 100, # in millisec
).add_to(m)
m

# Map ALL the buoys over time

# map it - download larger atlantic ocean - northern hemi - dataframe 

In [531]:
limits = { 'lat1': 12, 'lat2': 12+30, 'lon1': -63, 'lon2': -63+45 }

In [540]:
index_loader = ArgoIndexFetcher(cache=True)
#idx = index_loader.region([-80, -39, 0, 50, '2020-06-01', '2021-07'])
region_time = [limits['lon1'], limits['lon2'], limits['lat1'], limits['lat2'], '2020-06-01', '2021-07'] # up to june (exclusive last bound)
idx = index_loader.region(region_time)

df_buoy = idx.to_dataframe()
print("Current date and time: ", datetime.datetime.now())

list_buoy_ids = df_buoy.wmo.unique()
print(list_buoy_ids)
print(len(list_buoy_ids))

df_buoy = df_buoy.dropna(subset=['latitude', 'longitude'])
df_buoy.sort_values(by=['wmo', 'date'])
df_buoy.to_pickle(f'argoindex_{region_time}.pkl')

print("Current date and time: ", datetime.datetime.now())
df_buoy['longlat'] = list(zip(df_buoy.longitude, df_buoy.latitude)) 
df_timelapse = df_buoy[['longlat', 'date', 'date_update', 'wmo']]
display(df_timelapse)
print(limits)

Current date and time:  2021-06-12 22:12:07.870251
[4902441 6901271 3901686 6903720 4902344 4903048 3901841 1901712 6903715
 4902112 3901970 4903246 1902207 6901145 4901592 1902184 3901989 4903244
 4903218 3901603 4903055 7900541 6901256 6902766 4901798 4902339 4903215
 4902919 6901146 7900559 6902878 6901206 4901630 1901694 6902657 4903217
 6902659 4902340 4902100 3901236 4903059 5904664 3901597 3901659 6901142
 7900561 3901987 4903052 4902354 4902104 3901952 4902921 4903214 6901672
 3901681 6902714 3901601 3901622 6901254 7900524 4902455 7900542 3901682
 4903216 6901196 6902837 3901658 3901857 3901838 3901971 3901683 4902108
 6903716 6902663 6902655 6901144 6901195 4901587 4903054 3901654 3901635
 4902114 5904174 4902393 4902394 4903221 3901972 4902910 4901701 3901684
 4903219 6901143 6901213 4903245 3901839 7900560 5904669 3901640 6901922
 3901953 6902664 4903057 6902711 4903058 3901637 4902412 4902470 4903225
 6901258 3901687 7900543 4901699 1902209 3901845 6902914 3901951 4901623


,longlat,date,date_update,wmo
0,"(-60.976, 41.944)",2020-06-01 05:35:00,2020-11-24 16:53:09,4902441
1,"(-22.054, 26.713)",2020-06-01 05:39:00,2020-06-02 13:20:15,6901271
2,"(-55.053, 16.864)",2020-06-01 05:48:20,2021-03-17 07:23:49,3901686
3,"(-61.636, 25.974)",2020-06-01 06:41:53,2020-09-11 21:04:37,6903720
4,"(-55.767, 39.144)",2020-06-01 09:54:16,2020-06-11 11:00:29,4902344
...,...,...,...,...
6987,"(-29.783, 38.99)",2021-06-12 13:00:04,2021-06-12 17:00:43,4902347
6988,"(-50.222, 29.953)",2021-06-12 15:21:19,2021-06-12 19:00:37,4901721
6989,"(-33.192, 20.874)",2021-06-12 15:42:29,2021-06-12 19:00:44,4903218
6990,"(-33.726, 14.812)",2021-06-12 16:21:12,2021-06-12 18:12:29,3901682


{'lat1': 12, 'lat2': 42, 'lon1': -63, 'lon2': -18}


# map it - calculate grid heatmap counts

In [541]:
TIME_START = '2021-01-01'
TIME_END = '2021-01-31'

In [550]:
timed_df = df_buoy[ (df_buoy.date >= TIME_START) & (df_buoy.date <= TIME_END)]
# DO NOT USE DATE_UPDATE!
print(timed_df.shape)

(549, 12)


In [551]:
display(timed_df)

,file,date,longitude,latitude,ocean,profiler_code,institution_code,date_update,wmo,institution,profiler,longlat
3994,bodc/6901195/profiles/R6901195_139.nc,2021-01-01 00:26:34,-58.130,29.757,A,846,BO,2021-01-01 02:12:42,6901195,"BODC, United Kingdom","Webb Research, Seabird sensor","(-58.13, 29.757)"
3995,aoml/4903329/profiles/R4903329_007.nc,2021-01-01 02:31:05,-43.479,34.772,A,854,AO,2021-01-11 01:00:49,4903329,"AOML, USA",S2A float,"(-43.479, 34.772)"
3996,bodc/6903752/profiles/R6903752_002.nc,2021-01-01 03:08:34,-49.552,23.988,A,877,BO,2021-03-14 22:35:00,6903752,"BODC, United Kingdom",Unknown,"(-49.552, 23.988)"
3997,bodc/6903755/profiles/R6903755_003.nc,2021-01-01 03:29:58,-59.277,25.653,A,877,BO,2021-03-15 17:09:37,6903755,"BODC, United Kingdom",Unknown,"(-59.277, 25.653)"
3998,meds/4902500/profiles/R4902500_009.nc,2021-01-01 04:35:00,-54.338,39.805,A,844,ME,2021-01-01 01:15:34,4902500,"MEDS, Canada","Arvor, Seabird conductivity sensor","(-54.338, 39.805)"
...,...,...,...,...,...,...,...,...,...,...,...,...
4538,bodc/6903752/profiles/R6903752_005.nc,2021-01-30 15:36:34,-48.401,24.295,A,877,BO,2021-03-14 22:36:15,6903752,"BODC, United Kingdom",Unknown,"(-48.401, 24.295)"
4539,aoml/5904669/profiles/D5904669_188.nc,2021-01-30 16:28:34,-52.557,24.328,A,846,AO,2021-04-26 10:52:38,5904669,"AOML, USA","Webb Research, Seabird sensor","(-52.557, 24.328)"
4540,bodc/3901971/profiles/R3901971_093.nc,2021-01-30 16:38:30,-47.896,31.574,A,844,BO,2021-01-30 19:08:51,3901971,"BODC, United Kingdom","Arvor, Seabird conductivity sensor","(-47.896, 31.574)"
4541,aoml/4903329/profiles/R4903329_010.nc,2021-01-30 18:30:59,-44.538,34.749,A,854,AO,2021-02-09 19:42:19,4903329,"AOML, USA",S2A float,"(-44.538, 34.749)"


In [560]:

# http://www.garagegames.com/community/blogs/view/309
#lower_left = np.array([20, -70])
#diff = 21
#upper_right = lower_left + diff

#lower_left = np.array([12, -63])
#upper_right = lower_left + np.array([30, 45])
print(limits)

n_lat = 10 # create 3x3 grid, so 30/3, 45/3
n_lon = 15

lat_steps = np.linspace(limits['lat1'], limits['lat2'], n_lat + 1)
lon_steps = np.linspace(limits['lon1'], limits['lon2'], n_lon + 1)
print(lat_steps)
#print(lon_steps)

lat_stride = lat_steps[1] - lat_steps[0]
lon_stride = lon_steps[1] - lon_steps[0]
print(lat_stride, lon_stride)

zeros = [] # used for AABB(axis aligned bounding box) line intersection check later

boxes_sanity_check = []
profile_counts_grid = []
for lat in lat_steps[:-1]:
    for lon in lon_steps[:-1]:
        boxes_sanity_check.append((lat, lon))
        itfits = timed_df[ 
            (timed_df['latitude'] >= lat ) &
                 (timed_df['latitude'] < lat + lat_stride) &
            (timed_df['longitude'] >= lon ) &
                 (timed_df['longitude'] < lon + lon_stride) 
               ]
        print(itfits.shape, lat, lon)
        profile_counts_grid.append(itfits.shape[0])
        if itfits.shape[0] == 0: # There were no profiles in this lat long
            #zeros.append((lat, lat+lat_stride, lon, lon+lon_stride))
            zeros.append( shapely.geometry.box(lat, lat+lat_stride, lon, lon+lon_stride))
        

{'lat1': 12, 'lat2': 42, 'lon1': -63, 'lon2': -18}
[12. 15. 18. 21. 24. 27. 30. 33. 36. 39. 42.]
3.0 3.0
(3, 12) 12.0 -63.0
(7, 12) 12.0 -60.0
(3, 12) 12.0 -57.0
(0, 12) 12.0 -54.0
(0, 12) 12.0 -51.0
(11, 12) 12.0 -48.0
(5, 12) 12.0 -45.0
(0, 12) 12.0 -42.0
(0, 12) 12.0 -39.0
(6, 12) 12.0 -36.0
(3, 12) 12.0 -33.0
(5, 12) 12.0 -30.0
(0, 12) 12.0 -27.0
(6, 12) 12.0 -24.0
(0, 12) 12.0 -21.0
(0, 12) 15.0 -63.0
(3, 12) 15.0 -60.0
(0, 12) 15.0 -57.0
(1, 12) 15.0 -54.0
(3, 12) 15.0 -51.0
(3, 12) 15.0 -48.0
(0, 12) 15.0 -45.0
(0, 12) 15.0 -42.0
(0, 12) 15.0 -39.0
(4, 12) 15.0 -36.0
(6, 12) 15.0 -33.0
(6, 12) 15.0 -30.0
(7, 12) 15.0 -27.0
(15, 12) 15.0 -24.0
(3, 12) 15.0 -21.0
(2, 12) 18.0 -63.0
(1, 12) 18.0 -60.0
(6, 12) 18.0 -57.0
(4, 12) 18.0 -54.0
(5, 12) 18.0 -51.0
(8, 12) 18.0 -48.0
(9, 12) 18.0 -45.0
(4, 12) 18.0 -42.0
(0, 12) 18.0 -39.0
(9, 12) 18.0 -36.0
(1, 12) 18.0 -33.0
(6, 12) 18.0 -30.0
(3, 12) 18.0 -27.0
(2, 12) 18.0 -24.0
(13, 12) 18.0 -21.0
(8, 12) 21.0 -63.0
(4, 12) 21.0 -60.0

In [553]:
timed_df['latitude']

3994    29.757
3995    34.772
3996    23.988
3997    25.653
3998    39.805
         ...  
4538    24.295
4539    24.328
4540    31.574
4541    34.749
4542    33.595
Name: latitude, Length: 549, dtype: float64

# map it - init folium map 

In [554]:
m = folium.Map(zoom_start = 2, location=[30, -60], width=800, height=400)
# ULeft, URight, LLeft, LRight

## map it - then create gridded heatmap using counts 

In [574]:
print(coord_grids[0])
print(len(coord_grids))
print(len(boxes_sanity_check))
print(list(zip(boxes_sanity_check, coord_grids)))

{'lat': (42.0, 39.0), 'long': (-63.0, -60.0)}
150
150
[((12.0, -63.0), {'lat': (42.0, 39.0), 'long': (-63.0, -60.0)}), ((12.0, -60.0), {'lat': (42.0, 39.0), 'long': (-60.0, -57.0)}), ((12.0, -57.0), {'lat': (42.0, 39.0), 'long': (-57.0, -54.0)}), ((12.0, -54.0), {'lat': (42.0, 39.0), 'long': (-54.0, -51.0)}), ((12.0, -51.0), {'lat': (42.0, 39.0), 'long': (-51.0, -48.0)}), ((12.0, -48.0), {'lat': (42.0, 39.0), 'long': (-48.0, -45.0)}), ((12.0, -45.0), {'lat': (42.0, 39.0), 'long': (-45.0, -42.0)}), ((12.0, -42.0), {'lat': (42.0, 39.0), 'long': (-42.0, -39.0)}), ((12.0, -39.0), {'lat': (42.0, 39.0), 'long': (-39.0, -36.0)}), ((12.0, -36.0), {'lat': (42.0, 39.0), 'long': (-36.0, -33.0)}), ((12.0, -33.0), {'lat': (42.0, 39.0), 'long': (-33.0, -30.0)}), ((12.0, -30.0), {'lat': (42.0, 39.0), 'long': (-30.0, -27.0)}), ((12.0, -27.0), {'lat': (42.0, 39.0), 'long': (-27.0, -24.0)}), ((12.0, -24.0), {'lat': (42.0, 39.0), 'long': (-24.0, -21.0)}), ((12.0, -21.0), {'lat': (42.0, 39.0), 'long': (-2

In [555]:
coord_grids, boxes = get_geojson_grid(upper_right, lower_left , n_lat=10, n_lon=15) 

colormap = branca.colormap.linear.YlOrRd_09.scale(0, 30)
colormap = colormap.to_step(index=[0, 1, 2, 5, 10, 15, 20, 25, 30])
colormap.caption = 'Number of profiles'
colormap.add_to(m)

for counts, geo_json in zip(profile_counts_grid, boxes):

    if counts == 0:
        color = 'b'
        color = mpl.colors.to_hex(color)
    else:
        #color = plt.cm.Oranges(counts / len(boxes))
        #color = mpl.colors.to_hex(color)
        color = colormap(counts)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, 
                        color=color: {
                            'fillColor': color,
                            'color':"black",
                            'weight': 0.4,
                            'dashArray': '5, 5',
                            'fillOpacity': 0.60,
                        })
    popup = folium.Popup(f"Counts:{counts}")
    gj.add_child(popup)

    m.add_child(gj)
    
folium.map.Marker(
    [42,- 18],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt"><b>Heatmap</b><br>Num. of profiles per 3°x3°<br>From <font color="purple">{TIME_START}</font> to '
        f'<font color="purple">{TIME_END}</font><br><font color="blue">Blue is 0 counts</font> </div>',
        )
    ).add_to(m)

m

# Then Create Lines of All the buoys, to add to map

In [556]:
features = []
#cmap = plt.cm.get_cmap("tab20")#, len(list_gulf_of_mexico))
num_colors = len(list_buoy_ids)
cmap = matplotlib.cm.get_cmap('prism', num_colors)
NaT_fill = 0 
gps_fill = -99
    

for ith_buoy, buoy in enumerate(list_buoy_ids):
    # Create timelapse lines for a single buoy
    buoy_df = df_timelapse[ (df_timelapse.wmo == buoy) & (df_timelapse.date > TIME_START) & (df_timelapse.date_update < TIME_END)]
    # Create lines consist of 
    # (latlong_pointA), timeA to (latlong_pointB, timeB)
    buoy_df['line_coords'] = list(zip(
        buoy_df['longlat'].shift(fill_value = gps_fill), 
        buoy_df['longlat']))
    
    buoy_df = buoy_df.sort_values(by=['wmo','date'])
    
    buoy_df['str_date'] = buoy_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    buoy_df['line_times'] = list(zip(
        buoy_df['str_date'].shift(fill_value = NaT_fill) , 
        buoy_df['str_date']
    ))
    
    for _, row in buoy_df.iterrows():
        if row.line_times[0] == 0:
            # Due to use of pd.shift, first line is N/A 
            #print('na detected')
            continue
        #print('This buoy is ', ith_buoy, row.wmo,  row.line_times, row.line_coords)
        color = matplotlib.colors.rgb2hex(cmap(ith_buoy/num_colors))
        feature =  { "type": "Feature", 
                    "geometry": { "type": "LineString", 
                                 "coordinates": row.line_coords, },
                    "properties": { "times": row.line_times, 
                                   "style": { "color": color, "weight": 3}, 
                                   'icon': 'circle',
                                   'iconstyle':{
                                       'fillColor': color,
                                       'opacity': 0.9,
                                       'stroke': False,
                                       'radius':5 
                                   },
                                   # Add click-to-popup 
                                   'popup': f'Buoy ID: {row.wmo}, \n Current Date: {row.date} \n\n'
                                            f'<i>Line Details: {list(zip(row.line_times, row.line_coords))}</i>',#, Date uploaded: {row.date_update}',
                                  },
                   }
        features.append(feature)
#features

In [557]:
# Add buoy line features to map

In [558]:
# https://github.com/python-visualization/folium/blob/master/folium/plugins/timestamped_geo_json.py
# https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/
# https://www.linkedin.com/pulse/visualizing-nyc-bike-data-interactive-animated-maps-folium-toso
# https://python-visualization.github.io/folium/modules.html

#m = folium.Map(location=start_longlat, zoom_start=4)#, width=800, height=300)
plugins.TimestampedGeoJson(
    { "type": "FeatureCollection", "features": features},
    period="P1D", add_last_point=True, 
    duration = 'P2M',
    min_speed = 1,
    max_speed = 50,
    transition_time = 100, # in millisec
).add_to(m)


In [559]:
m

In [316]:
m.save('3x3_heatmap_with_buoylines.html')